In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter 
from scipy import stats
import geopandas as gpd
from shapely.geometry import Point
import folium
import re

In [68]:
hub = pd.read_excel("data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx", skiprows=1)

In [69]:
# #rename the columns
hub.columns = [y.lower() for y in [x.strip(' ') if x[-1:] == " " else re.sub(r'\W', '_', x) for x in hub.columns]]

In [70]:
metro = pd.read_csv("data/hubNashville__311__Service_Requests.csv", low_memory=False)

In [71]:
metro.columns = ['request_number' if "#" in x else re.sub(r'\W+', '_', x).lower() for x in metro.columns]

In [72]:
#This fuction works but it is time consuming 
# inProj = Proj(init='epsg:3857')
# outProj = Proj(init='epsg:4326')
# [transform(x, y) for x in hub.state_plan_x for y in hub.state_plan_y]

In [73]:
hub_geo = pd.merge(hub, metro[['request_number', 'latitude', 'longitude']], how='inner', on='request_number')
hub_geo = hub_geo.drop(['state_plan_x', 'state_plan_y'], axis=1)
hub_geo.loc[:, ('geometry')] = hub_geo.apply(lambda x: Point(x.longitude, x.latitude), axis=1)
hub_geo = gpd.GeoDataFrame(hub_geo, crs = {'init': 'epsg:4326'}, geometry = hub_geo['geometry'])

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,latitude,longitude,geometry
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,36.217300,-86.815400,POINT (-86.8154 36.2173)
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,36.213480,-86.838110,POINT (-86.83811 36.21348)
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,36.142930,-86.885570,POINT (-86.88557 36.14293)
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,36.212660,-86.789180,POINT (-86.78918 36.21266)
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,36.154870,-86.875000,POINT (-86.875 36.15487)
...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01 16:03:00,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,36.069130,-86.633970,POINT (-86.63396999999999 36.06913)
20222,267126,2019-11-01 16:08:00,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,36.169455,-86.741186,POINT (-86.74118559999999 36.1694553)
20223,267130,2019-11-01 16:13:00,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,36.185644,-86.671752,POINT (-86.67175220000001 36.18564399999999)
20224,267134,2019-11-01 16:15:00,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,36.054583,-86.619823,POINT (-86.619823 36.054583)


,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,latitude,longitude,geometry
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,36.21730,-86.81540,POINT (-86.8154 36.2173)
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,36.21348,-86.83811,POINT (-86.83811 36.21348)
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,36.14293,-86.88557,POINT (-86.88557 36.14293)
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,36.21266,-86.78918,POINT (-86.78918 36.21266)
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,36.15487,-86.87500,POINT (-86.875 36.15487)


In [67]:
hub_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20226 entries, 0 to 20225
Data columns (total 12 columns):
request_number      20226 non-null int64
date_opened         20226 non-null datetime64[ns]
request             20226 non-null object
description         20197 non-null object
incident_address    20217 non-null object
zip_code            20151 non-null float64
trash_hauler        19325 non-null object
trash_route         19279 non-null object
council_district    20177 non-null float64
latitude            20201 non-null float64
longitude           20201 non-null float64
geometry            20226 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 2.0+ MB
